In [1]:
import numpy as np
import pandas as pd
import gzip
import ast
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
path= 'C:/Users/usuario/Desktop/Proyecto_Individual_1/materiales/users_items.json.gz'

In [3]:
# en primera instancia creamos un archivo .txt para realizar una vista preliminar de como son nuestros datos
with gzip.open(path, 'rt', encoding= 'utf-8') as dec:
    with open('users_items.txt','w', encoding= 'utf-8', newline= '') as f:
        for linea in dec:
            f.write(linea)

In [4]:
# 
listas= []

with gzip.open(path, 'rt', encoding= 'utf-8') as file:
    for line in file:
        listas.append(ast.literal_eval(line))


In [5]:
# creamos el dataframe con nustros datos
df_users_items= pd.DataFrame(listas)
df_users_items.tail()

,user_id,items_count,steam_id,user_url,items
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."
88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,[]


In [6]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
# vemos que los registros no contienen valores nan

In [8]:
# eliminamos los registros que contienen todos sus valores nan y reiniciamos el indice de nuestro dataframe
# esto nos permite tanto optimizar nuestros datos, como reducir el espacio que ocupan
df_users_items= df_users_items.dropna(how= 'all').reset_index(drop= True)

In [9]:
df_users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [10]:
df_user_items2 = pd.json_normalize(listas, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'])

In [11]:
df_user_items2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


In [13]:
df_user_items2.head()

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [ ]:
# eliminamos registros duplicados, y tambien eliminamos la columna 'playtime_2weeks', ya que no nos brinda informacion reelevante para nuestro analisis, y el tiempo de jeugo ya lo tenemos en la columna 'playtime_forever'
df_user_items2 = df_user_items2.drop_duplicates()
df_user_items2 = df_user_items2.drop(columns= 'playtime_2weeks', inplace = True)

In [12]:
# generamos un archivo .csv 

df_user_items2.to_csv('ETL1_users_items.csv', index= False)

In [ ]:
# guardamos el dataframe en un archivo .parquet
pq.write_table(pa.Table.from_pandas(df_user_items2), '../data/ETL1_users_items.parquet')